# Running VisionTS models on BOOM benchmark

This notebook is adapted from the [GiftEval repository](https://github.com/SalesforceAIResearch/gift-eval/tree/main/notebooks) and shows how to run the VisionTS on the BOOM benchmark.

Make sure you download the BOOM benchmark and set the `BOOM` environment variable correctly before running this notebook.

We will use the `Dataset` class from GiftEval to load the data and run the model.

Download BOOM datasets. Calling `download_boom_benchmark` also sets the `BOOM` environment variable with the correct path, which is needed for running the evals below.

In [ ]:
import os
import json
from dotenv import load_dotenv
from dataset_utils import download_boom_benchmark

boom_path = "ChangeMe"
download_boom_benchmark(boom_path)
load_dotenv()

dataset_properties_map = json.load(open("../dataset_properties.json"))
all_datasets = list(dataset_properties_map.keys())
print(len(all_datasets))

In [ ]:
from gluonts.ev.metrics import (
    MAE,
    MAPE,
    MASE,
    MSE,
    MSIS,
    ND,
    NRMSE,
    RMSE,
    SMAPE,
    MeanWeightedSumQuantileLoss,
)

# Instantiate the metrics
metrics = [
    MSE(forecast_type="mean"),
    MSE(forecast_type=0.5),
    MAE(),
    MASE(),
    MAPE(),
    SMAPE(),
    MSIS(),
    RMSE(),
    NRMSE(),
    ND(),
    MeanWeightedSumQuantileLoss(quantile_levels=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
]

In [ ]:
import math
import sys
import os
import argparse
sys.path.append("../")

import einops
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from gluonts.model.forecast import SampleForecast
from gluonts.ev.metrics import MAE, MASE, MSE, ND, NRMSE, SMAPE
from gluonts.model.evaluation import evaluate_forecasts
from visionts import VisionTS, freq_to_seasonality_list

POSSIBLE_SEASONALITIES = {
    "S": [3600],  # 1 hour
    "T": [1440, 10080],  # 1 day or 1 week
    "H": [24],  # 1 day or 1 week
    "D": [7, 30, 365],  # 1 week, 1 month or 1 year
    "W": [52, 4], # 1 year or 1 month
    "M": [12, 6, 3], # 3 months, 6 months or 1 year
    "B": [5],
    "Q": [4, 2], # 6 months or 1 year
}

def imputation_nan(array):
    """
    Impute missing value using Naive forecasting.
    """
    not_nan_mask = ~np.isnan(array)
    if not_nan_mask.all():
        return array
    if not not_nan_mask.any():
        return np.zeros_like(array)

    array_imputed = np.copy(array)
    for i in range(len(array)):
        if not not_nan_mask[i]:
            array_imputed[i] = array_imputed[i - 1]
    return array_imputed


def forecast(model: VisionTS, train_list: list, test_list: list, batch_size, device, periodicity):
    # We combine testing data with the context lengths
    seq_len_to_group_data = {}
    for i in range(len(train_list)):
        train_len = len(train_list[i])
        if train_len not in seq_len_to_group_data:
            seq_len_to_group_data[train_len] = [[], [], []] # index, input, output
        seq_len_to_group_data[train_len][0].append(i)
        seq_len_to_group_data[train_len][1].append(train_list[i])
        seq_len_to_group_data[train_len][2].append(test_list[i])
    
    forecast_np = {} # raw index -> forecast
    for train_len in seq_len_to_group_data:
        cur_idx_list, cur_train, cur_test = seq_len_to_group_data[train_len]
        convert = lambda array: torch.FloatTensor(
            einops.rearrange(np.array(array), 'b t -> b t 1')
        ).to(device)
        cur_train = convert(cur_train)
        cur_test = convert(cur_test)
        context_len = cur_train.shape[1]
        pred_len = cur_test.shape[1]
        model.update_config(context_len=context_len, pred_len=pred_len, periodicity=periodicity)

        for batch_i in range(int(math.ceil(len(cur_train) / batch_size))):
            batch_start = batch_i * batch_size
            if batch_start >= len(cur_train):
                continue
            batch_end = batch_start + batch_size
            if batch_end > len(cur_train):
                batch_end = len(cur_train)

            cur_batch_train = cur_train[batch_start:batch_end]
            cur_batch_pred = model(cur_batch_train, fp64=True) # [b t 1]
            for i in range(len(cur_batch_pred)):
                cur_idx = cur_idx_list[batch_start + i]
                assert cur_idx not in forecast_np
                forecast_np[cur_idx] = cur_batch_pred[i, :, 0].detach().cpu().numpy()
    return np.stack([forecast_np[i] for i in range(len(train_list))])


def convert_context_len(context_len, no_periodicity_context_len, periodicity):
    if periodicity == 1:
        context_len = no_periodicity_context_len
    # Round context length to the integer multiples of the period
    context_len = int(round(context_len / periodicity)) * periodicity
    return context_len

In [ ]:
class visionts_wrapper:
    def __init__(self, context_len, no_periodicity_context_len, freq, device="cuda:0", checkpoint_dir="./ckpt", mae_arch="mae_base", batch_size=5, periodicity="autotune",):
        self.batch_size = batch_size
        self.model = VisionTS(mae_arch, ckpt_dir=checkpoint_dir).to(device)
        self.periodicity = periodicity
        self.device = device
        self.freq = freq
        self.context_len = context_len
        self.no_periodicity_context_len=no_periodicity_context_len
        
    def predict(self, test_data):
        data_train = [imputation_nan(x['target']) for x in test_data.input]
        data_test = [x['target'] for x in test_data.label]
        pred_len = len(data_test[0])

        if self.periodicity == "autotune":
            seasonality_list = freq_to_seasonality_list(self.freq, POSSIBLE_SEASONALITIES)
            best_valid_mae = float('inf')
            best_valid_p = 1
            for periodicity in tqdm(seasonality_list, desc='validate seasonality'):
                cur_context_len = convert_context_len(self.context_len, self.no_periodicity_context_len, periodicity)
    
                val_train = [x[-cur_context_len-pred_len:-pred_len] for x in data_train]
                val_test = [x[-pred_len:] for x in data_train]
                val_forecast = forecast(self.model, val_train, val_test, self.batch_size, self.device, periodicity)
                val_mae = np.abs(np.asarray(val_test) - val_forecast).mean()
                if val_mae < best_valid_mae:
                    best_valid_p = periodicity
                    best_valid_mae = val_mae
                    tqdm.write(f"autotune: P = {periodicity} | valid mae = {val_mae}, accept!")
                else:
                    tqdm.write(f"autotune: P = {periodicity} | valid mae = {val_mae}, reject!")
            periodicity = best_valid_p
            print("finishing autotuning, best period =", periodicity)
        elif self.periodicity == "freq":
            periodicity = freq_to_seasonality_list(self.freq, POSSIBLE_SEASONALITIES)[0]
        else:
            periodicity = int(self.periodicity)

        cur_context_len = convert_context_len(self.context_len, self.no_periodicity_context_len, periodicity)
        train = [x[-cur_context_len:] for x in data_train]
        forecast_values = forecast(self.model, train, data_test, self.batch_size, self.device, periodicity)
        # print(forecast_values.shape)
        sample_forecasts = []
        for item, ts in zip(forecast_values, test_data.input):
            forecast_start_date = ts["start"] + len(ts["target"])
            sample_forecasts.append(
                SampleForecast(
                    samples=np.reshape(item, (1, -1)), start_date=forecast_start_date
                )
            )

        metrics_df = evaluate_forecasts(
            sample_forecasts,
            test_data=test_data,
            metrics=metrics,
        )
        return metrics_df

## Evaluation

Now that we have our predictor class, we can use it to predict on the boom benchmark datasets. We will use the `evaluate_model` function from `gluonts` to evaluate the model. We are going to store the results in a csv file called `all_results.csv` under the `results/visionts` folder.

The first column in the csv file is the dataset config name which is a combination of the dataset name, frequency and the term:

```python
f"{dataset_name}/{freq}/{term}"
```

In [ ]:
from gluonts.model import evaluate_model
import csv
import os
import time
from gluonts.time_feature import get_seasonality
from gift_eval.data import Dataset
import torch

torch.set_float32_matmul_precision("high")

# Iterate over all available datasets
model_name = "visionts"
output_dir = f"ChangeMe/{model_name}/"
# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Define the path for the CSV file
csv_file_path = os.path.join(output_dir, "all_results.csv")

with open(csv_file_path, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)

    # Write the header
    writer.writerow(
        [
            "dataset",
            "model",
            "eval_metrics/MSE[mean]",
            "eval_metrics/MSE[0.5]",
            "eval_metrics/MAE[0.5]",
            "eval_metrics/MASE[0.5]",
            "eval_metrics/MAPE[0.5]",
            "eval_metrics/sMAPE[0.5]",
            "eval_metrics/MSIS",
            "eval_metrics/RMSE[mean]",
            "eval_metrics/NRMSE[mean]",
            "eval_metrics/ND[0.5]",
            "eval_metrics/mean_weighted_sum_quantile_loss",
            "domain",
            "num_variates",
            "dataset_size",
        ]
    )

for ds_num, ds_name in enumerate(all_datasets):
    print(f"Processing dataset: {ds_name} ({ds_num + 1} of {len(all_datasets)})")
    dataset_term = dataset_properties_map[ds_name]["term"]
    terms = ["short", "medium", "long"]
    for term in terms:
        if (term == "medium" or term == "long") and dataset_term == "short":
            continue
        ds_freq = dataset_properties_map[ds_name]["frequency"]
        ds_config = f"{ds_name}/{ds_freq}/{term}"

        # Initialize the dataset, since Moirai support multivariate time series forecast, it does not require
        # to convert the original data into univariate
        to_univariate = False if Dataset(name=ds_name, term=term,to_univariate=False, storage_env_var="BOOM").target_dim == 1 else True
        dataset = Dataset(name=ds_name, term=term, to_univariate=to_univariate, storage_env_var="BOOM")
        dataset_size = len(dataset.test_data)
        print(f"Dataset size: {dataset_size}")

        model = visionts_wrapper(2000, 2000, dataset.freq[0], periodicity=1)
        season_length = get_seasonality(dataset.freq)
        res = model.predict(dataset.test_data)
        # Append the results to the CSV file
        with open(csv_file_path, "a", newline="") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(
                [
                    ds_config,
                    "visonts",
                    res["MSE[mean]"][0],
                    res["MSE[0.5]"][0],
                    res["MAE[0.5]"][0],
                    res["MASE[0.5]"][0],
                    res["MAPE[0.5]"][0],
                    res["sMAPE[0.5]"][0],
                    res["MSIS"][0],
                    res["RMSE[mean]"][0],
                    res["NRMSE[mean]"][0],
                    res["ND[0.5]"][0],
                    res["mean_weighted_sum_quantile_loss"][0],
                    dataset_properties_map[ds_name]["domain"],
                    dataset_properties_map[ds_name]["num_variates"],
                    dataset_size,
                ]
            )

        print(f"Results for {ds_name} have been written to {csv_file_path}")

## Results

Running the above cell will generate a csv file called `all_results.csv` under the `results/visonts` folder containing the results for the Timer model on the boom benchmark. The csv file will look like this:

In [ ]:
import pandas as pd
df = pd.read_csv(output_dir + "/all_results.csv")
df